## Importação de Bibliotecas

In [16]:
# Importando bibliotecas
import pandas as pd
import os 
import warnings
import folium
import openrouteservice
import ast

In [17]:
#Ajustes
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidt", None)
warnings.filterwarnings("ignore")

## Importando os dados

In [18]:
custumers = pd.read_csv("Dados/custumers.csv", sep=",", encoding="utf-8")
plant = pd.read_csv("Dados/plant.csv", sep=",", encoding="utf-8")



In [19]:
#Visualizando os dados
plant

,id_custumer,plant_id,plant_name,Latitude,Longitude
0,2464,SP001,Sorocaba,-23.43386,-47.39249
1,2467,SP001,Sorocaba,-23.43386,-47.39249
2,2488,SP001,Sorocaba,-23.43386,-47.39249
3,2491,SP001,Sorocaba,-23.43386,-47.39249
4,2494,SP001,Sorocaba,-23.43386,-47.39249
...,...,...,...,...,...
430,5272,SP001,Sorocaba,-23.43386,-47.39249
431,5350,SP001,Sorocaba,-23.43386,-47.39249
432,5353,SP001,Sorocaba,-23.43386,-47.39249
433,5356,SP001,Sorocaba,-23.43386,-47.39249


In [20]:
custumers

,id_custumer,custumer_name,City,Latitude,Longitude
0,2431,CUSTOMER 2431,Alagoinhas,-12.103680,-38.355090
1,2434,CUSTOMER 2434,Alfenas,-21.466445,-45.996509
2,2437,CUSTOMER 2437,Alfenas,-21.466855,-45.984828
3,2440,CUSTOMER 2440,Anápolis,-16.424773,-48.993149
4,2443,CUSTOMER 2443,Anápolis,-16.336820,-48.987640
...,...,...,...,...,...
995,5416,CUSTOMER 5416,Vitória,-20.300200,-40.292200
996,5419,CUSTOMER 5419,Vitória,-20.322200,-40.274200
997,5422,CUSTOMER 5422,Vitória,-20.299803,-40.292414
998,5425,CUSTOMER 5425,Vitória,-20.299906,-40.292401


In [21]:
#filtrando as a planta de Sorocaba com a latitude e longitude
filiais = plant[["plant_name", "Latitude", "Longitude"]]
filiais = filiais[filiais["plant_name"].isin(["Sorocaba"])].drop_duplicates().reset_index()
filiais

,index,plant_name,Latitude,Longitude
0,0,Sorocaba,-23.43386,-47.39249


## Função para filtrar Latitude e Longitude da Entrega

In [22]:
#Realizando o merge entre as tabelas para facilitar a manipulação dos dados
df = pd.merge(
    plant,
    custumers,
    on="id_custumer",
    how="inner"
)

In [23]:
df.sample(10)

,id_custumer,plant_id,plant_name,Latitude_x,Longitude_x,custumer_name,City,Latitude_y,Longitude_y
246,4519,SP001,Sorocaba,-23.43386,-47.39249,CUSTOMER 4519,Santo André,-23.702722,-46.555332
136,3493,SP001,Sorocaba,-23.43386,-47.39249,CUSTOMER 3493,Limeira,-22.634535,-47.398722
60,3106,SP001,Sorocaba,-23.43386,-47.39249,CUSTOMER 3106,Guará,-20.484733,-47.872097
181,3889,SP001,Sorocaba,-23.43386,-47.39249,CUSTOMER 3889,Piracicaba,-22.764380,-47.631770
42,2854,SP001,Sorocaba,-23.43386,-47.39249,CUSTOMER 2854,Cravinhos,-21.371478,-47.759932
283,4642,SP001,Sorocaba,-23.43386,-47.39249,CUSTOMER 4642,São Carlos,-22.036706,-47.952438
5,2497,SP001,Sorocaba,-23.43386,-47.39249,CUSTOMER 2497,Araraquara,-21.839200,-48.206200
141,3616,SP001,Sorocaba,-23.43386,-47.39249,CUSTOMER 3616,Monte Alto,-21.297040,-48.500950
40,2848,SP001,Sorocaba,-23.43386,-47.39249,CUSTOMER 2848,Cravinhos,-21.371200,-47.771200
34,2782,SP001,Sorocaba,-23.43386,-47.39249,CUSTOMER 2782,Capivari,-23.040769,-47.554816


In [24]:
df.dtypes

id_custumer        int64
plant_id          object
plant_name        object
Latitude_x       float64
Longitude_x      float64
custumer_name     object
City              object
Latitude_y       float64
Longitude_y      float64
dtype: object

In [25]:
#Função para encontrar e retornar latitude e longitude de cada cliente da entrega
def encontrar_long_lat_clientes(lista_clientes, cidade_escolhida):
    #Filtrando o dataframe
    clientes_filtrados = df[df["id_custumer"].isin(lista_clientes)]

    #Ordenando o dataframe na mesma ordem da lista de clientes 
    clientes_filtrados = clientes_filtrados.set_index("id_custumer").reindex(lista_clientes)

    #Retornando lista com tuplas de longitude e latitude de clientes
    long_lat = list(zip(clientes_filtrados["Longitude_y"], clientes_filtrados["Latitude_y"]))

    #Coordenadas de Sorocaba
    plant = {
        "Sorocaba": [(-47.39249,-23.43386)]
    }

    #incluindo itens na lista final, iniciando e terminando na cidade escolhida
    lista_final = plant[cidade_escolhida]
    lista_final.extend(long_lat)
    lista_final.append(plant[cidade_escolhida][0])

    return lista_final



In [35]:
#Testando a função, adicionar nome dos clientes na lista
lista_clientes = custumers[custumers["custumer_name"].isin(["CUSTOMER 3265", "CUSTOMER 2782", "CUSTOMER 4642"])]["id_custumer"]
coordenadas = encontrar_long_lat_clientes(lista_clientes, "Sorocaba")
coordenadas

[(-47.39249, -23.43386),
 (-47.554816, -23.040769),
 (-47.3162, -23.3102),
 (-47.952438, -22.036706),
 (-47.39249, -23.43386)]

## Conectando com API Open ROute Service

In [36]:

def calculando_rota(coordenadas):
    # Configurar chave API
    ORS_API_KEY = os.getenv("TOKEN_API") #Busca a API

    # Criar cliente OpenRouteService
    client = openrouteservice.Client(key=ORS_API_KEY)

    try:
        # Exemplo: calcular rota com as coordenadas já obtidas
        rota = client.directions(
            coordinates=coordenadas,
            profile="driving-car", 
            # profile="driving-hgv", 
            format_out="geojson"    
        )

        return rota
    
    except: 
        return "N/A"
 

In [37]:
#Calculando a rota mostrando distância e duração totais saindo de Sorocaba para as coordenadas dos clientes
rota = calculando_rota(coordenadas)

if rota is None:
    print("Falha ao calcular rota. Verifique coordenadas.")
else:
    distancia_total = 0
    duracao_total = 0
    for segment in rota["features"][0]["properties"]["segments"]:
        distancia_total += segment["distance"]
        duracao_total += segment["duration"]

    distancia_total_km = distancia_total / 1000
    duracao_total_min = duracao_total / 60

    print(f"Distância: {distancia_total_km:.2f} km")
    print(f"Duração: {duracao_total_min:.1f} minutos")


Distância: 501.09 km
Duração: 393.3 minutos


In [39]:
# Exemplo de entragas/Clientes 
lista_clientes = lista_clientes

# Buscando Coordenadas (retorna no formato ORS: (lon, lat))
coord_sorocaba = encontrar_long_lat_clientes(lista_clientes, "Sorocaba")

# Calculando rotas
rota_sorocaba = calculando_rota(coord_sorocaba)

# Criando o objeto mapa — convertendo (lon, lat) -> (lat, lon)
lat = coord_sorocaba[1][1]
lon = coord_sorocaba[1][0]

mapa = folium.Map(location=[lat, lon], zoom_start=10)

# Plotando rota
folium.GeoJson(
    rota_sorocaba, 
    name="Rota Ideal", 
    style_function=lambda x: {
        "color": "red",
        "weight": 4,
    }
).add_to(mapa)

# Adicionando marcadores
for lon, lat in coord_sorocaba:
    folium.Marker(
        location=[lat, lon],    # Folium = (lat, lon)
        icon=folium.Icon(color="red", icon="flag")
    ).add_to(mapa)

    mapa.save("mapa3.html")
